# A-) SETUP

In [1]:
# Cell 0a — Environment sanity: pin versions (RUN THIS FIRST, then *restart runtime*)
# This avoids the two errors in your notebook:
# 1) protobuf descriptor error in model_builder_tf2_test
# 2) NumPy binary mismatch coming via JAX

%pip uninstall -y jax jaxlib tensorflow-probability || true
%pip install -U pip
%pip install "numpy==1.26.4" "protobuf==3.20.3" "tf-slim" "Cython" "contextlib2" "lxml" "pillow" "matplotlib" "pandas" "opencv-python" "absl-py"

import sys, subprocess
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"

# TensorFlow choice based on Python version
if pyver.startswith("3.12"):
    # For Python 3.12, use TF 2.17+ wheels
    subprocess.run([sys.executable, "-m", "pip", "install", "tensorflow==2.17.*"], check=False)
else:
    # With Python 3.10/3.11, TF 2.13.1 is the most compatible with the TF Object Detection API repo
    subprocess.run([sys.executable, "-m", "pip", "install", "tensorflow==2.13.1"], check=False)

# Cell 0b — (Optional) Fallback for protobuf runtime
# If you still see "Descriptors cannot be created directly", *before* any imports do:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
print("Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python")

# clone models
!git clone https://github.com/tensorflow/models.git

# use protobuf to convert between languages
!apt-get install -y protobuf-compiler
!cd /content/models/research && protoc object_detection/protos/*.proto --python_out=.

# install the object detection API
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research/
!cd /content/models/research && pip install .

# Test installation (more robust way)
import os

# Change into research dir first
os.chdir("models/research")

# Compile protos again to be safe (works on Linux/Colab; on Windows use preinstalled protoc)
os.system("protoc object_detection/protos/*.proto --python_out=.")

# Run the test as a module to ensure correct PYTHONPATH
exit_code = os.system("python -m object_detection.builders.model_builder_tf2_test")
print("Exit code:", exit_code)

# Change back if needed
os.chdir("../..")


Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
Cloning into 'models'...
remote: Enumerating objects: 102718, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 102718 (delta 104), reused 72 (delta 65), pack-reused 102570 (from 4)
Receiving objects: 100% (102718/102718), 642.83 MiB | 27.00 MiB/s, done.
Resolving deltas: 100% (74173/74173), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.4).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Processing /content/models/research
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done

# B-) Model Preparing

In [2]:
# Cell 0 - Import Libraries
import os
import pathlib
import tensorflow as tf

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# Cell 1 - Enable GPU for dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# Cell 2 - Example Pictures
!wget -nc https://lazyprogrammer.me/course_files/object_detection_images/walkingdog.jpeg
!wget -nc https://lazyprogrammer.me/course_files/object_detection_images/traffic.jpeg
!wget -nc https://lazyprogrammer.me/course_files/object_detection_images/jungle.jpeg
!wget -nc https://lazyprogrammer.me/course_files/object_detection_images/intersection.jpeg
!ls

--2025-10-26 20:46:44--  https://lazyprogrammer.me/course_files/object_detection_images/walkingdog.jpeg
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195368 (191K) [image/jpeg]
Saving to: ‘walkingdog.jpeg’

walkingdog.jpeg     100%[===================>] 190.79K  --.-KB/s    in 0.04s   

2025-10-26 20:46:44 (5.16 MB/s) - ‘walkingdog.jpeg’ saved [195368/195368]

--2025-10-26 20:46:44--  https://lazyprogrammer.me/course_files/object_detection_images/traffic.jpeg
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160514 (157K) [image/jpeg]
Saving to: ‘traffic.jpeg’

traffic.jpeg        

In [5]:
# Cell 3 - Image Paths
IMAGE_PATHS = [
    'intersection.jpeg',
    'traffic.jpeg',
    'jungle.jpeg',
    'walkingdog.jpeg'
]

In [6]:
# Cell 4 - Download zoo object detection model and extract
#url = 'https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md'
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname  = 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin = url,
    untar  = True
)

PATH_TO_MODEL_DIR

386527459/386527459 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'